# Shocks Building | "The role of sufficiency measures in a decarbonizing Europe"

This notebook guides you through the process of building "shock" files

> **"The role of sufficiency measures in a decarbonizing Europe"**  
> Published: 29 April 2025, Ecological Economics  
> DOI: [10.1016/j.ecolecon.2025.108645](https://doi.org/10.1016/j.ecolecon.2025.108645)

---

## About the Zenodo Repository

The Zenodo repository contains all data and results related to the study.  
**[Zenodo Link](https://doi.org/10.5281/zenodo.15070606)**


---

## 1. Download Required Shock Files

The following cell downloads and extracts the ZIP file containing all necessary files for building shocks.  
**Note:** This step only needs to be run once.

In [ ]:
import requests
import os
import zipfile

url = "https://zenodo.org/records/15308822/files/Files%20for%20shocks.zip?download=1" 
output_path = "Data/Files for shocks.zip"
extract_path = "Data"

os.makedirs(os.path.dirname(output_path), exist_ok=True)
response = requests.get(url, stream=True)
with open(output_path, "wb") as f:
    for chunk in response.iter_content(chunk_size=8192):
        f.write(chunk)

# Extract the ZIP file
with zipfile.ZipFile(output_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)
    
    # Delete the ZIP file after extraction
    os.remove(output_path)


## 2. Download the MARIO Database (Optional)

If you did not run "1. Database building", you can download the MARIO database directly from Zenodo using the cell below.  
If you already have the database, you can skip this step.

In [ ]:
# In case you didn't run step 1. Database building, you can download the database from Zenodo. Otherwise, skip this cell and run the next one to directly parse the database
import requests
import os
import zipfile

url = "https://zenodo.org/records/15308822/files/MARIO%20database.zip?download=1" 
output_path = "Data/MARIO database.zip"
extract_path = "Data/MARIO database"

os.makedirs(os.path.dirname(output_path), exist_ok=True)
response = requests.get(url, stream=True)
with open(output_path, "wb") as f:
    for chunk in response.iter_content(chunk_size=8192):
        f.write(chunk)

# Extract the ZIP file
with zipfile.ZipFile(output_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)
    
    # Delete the ZIP file after extraction
    os.remove(output_path)

## 3. Parse the MARIO Database

This cell parses the MARIO database from the extracted files.  
It uses the `mario` package to load the Supply-Use Table (SUT) in "flows" mode.

In [ ]:
import mario
import warnings 
warnings.filterwarnings("ignore") # Pandas became a bit noisy with warnings

db = mario.parse_from_txt(
    path = "Data/MARIO database/flows", # it can be also directly downloaded from Zenodo, skipping step 1. Database building
    table = 'SUT', 
    mode ='flows',
    name = "world"
)

## 4. Build Shock Files

This section sets up the parameters for scenario, measure, and year.  
It then loops through all combinations, filling a template Excel file for each shock and saving the results in the `Shocks/filled_files copy/` directory.

- `measure`: List of measures (e.g., Diets, Sharing spaces, etc.)
- `background`: List of background scenarios (e.g., Reference, NZE, BAU)
- `years`: Years for which shocks are built (2011, 2020, ..., 2050)

The `ShockMap.xlsx` file contains the instructions for building each shock.  
The `get_shock_file` function fills the template for each scenario.

In [ ]:
from _support import other_parameters
from _support import get_shock_file
import pandas as pd

measure = [
    'All',  # All measures
    'D',    # Diets
    'S',    # Sharing spaces
    'M',    # Car efficiency
    'P',    # Sharing products
    'F',    # Flying less
    'B',    # Cycling more
    0,      # No measure
    ] 
background = [
    'REF',  # Reference scenario
    'NZE',  # Quicker transition
    'BAU'   # Business as usual
    ] 
years = [
    '2011',
    '2020',
    '2025',
    '2030',
    '2035',
    '2040',
    '2045',
    '2050',
    ] # 5 year steps from 2020 to 2050 (pass list of strings)

shockmap = pd.read_excel("Shocks/ShockMap.xlsx",index_col=[0,1,2,3,4,5],header=[0,1])  # This file contains all the instructions to build the shocks
db.get_shock_excel("Shocks/template.xlsx") # empty template that will be filled multiple times, once for each shock

for b in background:
    for m in measure:
        for y in years:

            template = pd.read_excel("Shocks/template.xlsx",sheet_name=None)

            shock_file = get_shock_file(
                b,
                m,
                y,
                shockmap,
                db,
                other_parameters,
                template
                )

            with pd.ExcelWriter(f'Shocks/filled_files/{b}_{m}_{y}.xlsx') as writer:
                for key in shock_file.keys():
                    shock_file[key].to_excel(writer, sheet_name=key,index=False)
